## Price-Only LSTM — Baseline (Henry Hub)

First Iteration of LSTM, goal is to produce an extendible BaseLine that forecasts **next day Henry Hub price** using **price history only**

### Step 1 — Configuration and Library Imports
Load paths, model parameters, and import required libraries.

In [1]:
SEED      = 1337

# ==== imports ====
import os, json, pickle, platform, math, sys, time
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone

np.random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import backend as K

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from optuna.importance import get_param_importances

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler

CSV = "../numeric_data/henryhub_master.csv"   # columns: date, price

FEATURES = ["price"]
TARGET = "price"

EXPERIMENT_NAME = "Price Only"
MODEL_TAG = "lstm"
RUN_TS = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")

TOP_K = 10
SAVED_RESULTS_DIR = "saved_results"

# run output dir: saved_results/<timestamp>_<experimentname>_lstm/
RUN_NAME = f"{RUN_TS}_{EXPERIMENT_NAME.replace(' ', '')}_{MODEL_TAG}"
RUN_DIR = os.path.join(SAVED_RESULTS_DIR, RUN_NAME)
TOPMODELS_DIR = os.path.join(RUN_DIR, "top_models")
TRIAL_SUMMARY_CSV = os.path.join(RUN_DIR, "trial_summary.csv")

# DB inside the run folder (so each run is self-contained)
DB_PATH = os.path.join(RUN_DIR, f"optuna_{EXPERIMENT_NAME.replace(' ', '')}_{MODEL_TAG}.db")

# study name includes timestamp + experiment + model
STUDY_NAME = RUN_NAME

os.makedirs(TOPMODELS_DIR, exist_ok=True)

/Users/adam/Documents/CS310 - 3rd Year Project/Baseline LSTM Model/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 2 - Load the pre-processed Henry Hub daily spot price dataset.

This file already has:
	•	a continuous business-day index
	•	one column of numeric prices
	•	no gaps or missing dates

This ensures the LSTM receives a clean, uniform time series.

In [2]:
# Load the business-day–filled price series
data = pd.read_csv(CSV, parse_dates=["date"])

# Ensure sorted order and clean index
data = data.sort_values("date").reset_index(drop=True)

print("Rows:", len(data))
data.head(), data.tail()

Rows: 4086


(        date  id  price  storage_bcf  production_bcf  usd_index  temp_c  \
 0 2010-01-04  HH   6.09       3117.0       1737233.0    92.3566    -4.2   
 1 2010-01-05  HH   6.19       3117.0       1737233.0    92.2236    -6.4   
 2 2010-01-06  HH   6.47       3117.0       1737233.0    92.0941    -2.2   
 3 2010-01-07  HH   7.51       3117.0       1737233.0    92.3684    -4.4   
 4 2010-01-08  HH   6.56       2850.0       1737233.0    92.1485    -8.3   
 
    temp_max_c  temp_min_c  
 0        -0.6        -7.8  
 1        -1.7       -11.1  
 2         2.8        -7.2  
 3         0.6        -9.4  
 4        -4.4       -12.2  ,
            date  id  price  storage_bcf  production_bcf  usd_index  temp_c  \
 4081 2025-08-26  HH   2.82       3217.0       3372895.0   120.9304   21.40   
 4082 2025-08-27  HH   2.88       3217.0       3372895.0   121.1236   19.20   
 4083 2025-08-28  HH   2.90       3217.0       3372895.0   120.6537   21.65   
 4084 2025-08-29  HH   2.88       3272.0       3372

### Step 3 — Chronological Split and Train-Only Scaling

In this step, the dataset is split into train, validation, and test sets in time order.

We then fit scalers on the training split only, and apply them to validation and test sets.

Guards check that each split has enough rows to build LSTM windows (LOOKBACK + HORIZON), and we verify that no NaNs remain.

In [3]:
TEST_FRAC = 0.15
VAL_FRAC  = 0.15
SCALER    = RobustScaler

# lookback: number of past days used as input for each LSTM sample (e.g., 60-day window)
LOOKBACK  = 60  

# horizon: how many days ahead we predict (e.g., 1-day-ahead forecast)
HORIZON   = 1   

def time_split_scale(
    data: pd.DataFrame,
    feature_cols: list,
    target_col: str,
    test_frac: float = TEST_FRAC,
    val_frac: float  = VAL_FRAC,
    scaler_cls = SCALER
):
    """Chronological split -> train/val/test, then train-only scaling."""
    data = data.sort_values("date").reset_index(drop=True).copy()
    n = len(data)

    n_test     = int(n * test_frac)
    n_trainval = n - n_test
    n_val      = int(n_trainval * val_frac)
    n_train    = n_trainval - n_val

    train_data = data.iloc[:n_train].copy()
    val_data   = data.iloc[n_train:n_train+n_val].copy()
    test_data  = data.iloc[n_train+n_val:].copy()

    if train_data[feature_cols + [target_col]].isna().any().any():
        raise ValueError("NaNs in train split; fix upstream before modeling.")

    X_train_raw = train_data[feature_cols].to_numpy(dtype="float32")
    y_train_raw = train_data[[target_col]].to_numpy(dtype="float32")
    X_val_raw   = val_data[feature_cols].to_numpy(dtype="float32")
    y_val_raw   = val_data[[target_col]].to_numpy(dtype="float32")
    X_test_raw  = test_data[feature_cols].to_numpy(dtype="float32")
    y_test_raw  = test_data[[target_col]].to_numpy(dtype="float32")

    X_scaler = scaler_cls().fit(X_train_raw)
    y_scaler = scaler_cls().fit(y_train_raw)

    X_train = X_scaler.transform(X_train_raw)
    y_train = y_scaler.transform(y_train_raw)
    X_val   = X_scaler.transform(X_val_raw)
    y_val   = y_scaler.transform(y_val_raw)
    X_test  = X_scaler.transform(X_test_raw)
    y_test  = y_scaler.transform(y_test_raw)

    return X_train, y_train, X_val, y_val, X_test, y_test, X_scaler, y_scaler, (train_data, val_data, test_data)

X_train_raw, y_train_raw, X_val_raw, y_val_raw, X_test_raw, y_test_raw, X_scaler, y_scaler, (train_data, val_data, test_data) = \
    time_split_scale(
        data=data,
        feature_cols=FEATURES,
        target_col=TARGET,
        test_frac=TEST_FRAC,
        val_frac=VAL_FRAC,
        scaler_cls=SCALER
    )

### Step 4 — Transform the time series into overlapping LSTM sequences.

Each sample uses a 60-day history window (t−59 … t) to predict the next value at t+1.

So each day has the previous 60 days of data to predict the price for next day.

In [4]:
def make_windows(X_raw, y_raw, lookback=LOOKBACK, horizon=HORIZON):
    """
    Convert the 1-D time series into overlapping LSTM training windows.

    For each index i:
        - X window  = the previous `lookback` values (shape: [lookback, n_features])
        - y target  = the value `horizon` steps ahead

    Example (lookback=60, horizon=1):
        Input window uses days [t-59 ... t]
        Target is the price on day t+1
    """

    X_seq, y_seq = [], []

    for i in range(len(X_raw) - lookback - horizon + 1):

        # Input: the 60-day history window starting at position i
        X_seq.append(X_raw[i : i + lookback])

        # Target: the next value after the window (1-day-ahead forecast)
        y_seq.append(y_raw[i + lookback + horizon - 1, 0])

    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.float32)


# Generate windows for train, validation, and test sets
X_train, y_train = make_windows(X_train_raw, y_train_raw)
X_val,   y_val   = make_windows(X_val_raw,   y_val_raw)
X_test,  y_test  = make_windows(X_test_raw,  y_test_raw)

X_train.shape, X_val.shape, X_test.shape

((2893, 60, 1), (461, 60, 1), (552, 60, 1))

### Model Hyperparameter Tuning

#### Parts we will Tune:
##### - Architecture
    • layers ∈ {1, 2, 3}
    • units (pyramids):
       1-layer: {32, 64, 128}
       2-layer: {(64,32), (96,48), (128,64)}
       3-layer: {(64,32,16), (32,16,8), (16,8,4)}
   • optional Dense: {0, 32} with activation {relu, tanh}
##### - Regularisation
   • dropout ∈ {0.0, 0.1, 0.2, 0.3}
   • recurrent_dropout ∈ {0.0, 0.05, 0.1, 0.15}
   • clipnorm ∈ {None, 1.0}
##### - Sequence windowing
   • lookback ∈ {30, 45, 60, 90, 120}
   • batch_size ∈ {32, 64, 128}
##### - Optimisation
   • learning_rate ~ log-uniform [2e-4, 2e-3]

In [5]:
def build_lstm_model_from_trial(trial, steps, feats):
    """
    Build a Keras LSTM according to the Optuna trial.
    """
    layers_n = trial.suggest_categorical("layers", [1, 2, 3])

    if layers_n == 1:
        lstm_units = [trial.suggest_categorical("units1", [32, 48, 64])]
    elif layers_n == 2:
        pair_choices = [(64, 48), (64, 32), (48, 32)]
        idx = trial.suggest_categorical("units2_idx", list(range(len(pair_choices))))
        lstm_units = list(pair_choices[idx])
    elif layers_n == 3:
        triple_choices = [(64, 32, 16), (32, 16, 8), (16, 8, 4)]
        idx = trial.suggest_categorical("units3_idx", list(range(len(triple_choices))))
        lstm_units = list(triple_choices[idx])
    else:
        raise ValueError(f"Unsupported layers_n={layers_n}")

    dropout = trial.suggest_categorical("dropout", [0.0, 0.05])
    rdrop = 0.0

    lr = trial.suggest_float("lr", 1e-4, 3e-3, log=True)
    opt = keras.optimizers.Adam(learning_rate=lr)

    m = keras.Sequential([layers.Input(shape=(steps, feats))])
    for i, u in enumerate(lstm_units):
        m.add(layers.LSTM(
            u,
            return_sequences=(i < len(lstm_units) - 1),
            dropout=dropout,
            recurrent_dropout=rdrop
        ))
    dense_units = 0
    m.add(layers.Dense(1))
    m.compile(optimizer=opt, loss="mse", metrics=[keras.metrics.MeanAbsoluteError(name="mae")])
    return m, lstm_units, lr, dropout, rdrop, dense_units


In [6]:
class ManualPruningCallback(keras.callbacks.Callback):
    """
    Reports val_loss to Optuna each epoch and prunes bad trials early.
    Stores prune diagnostics (epoch + last val_loss) in the Optuna DB (user_attrs).
    """
    def __init__(self, trial):
        super().__init__()
        self.trial = trial

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get("val_loss")
        if current is None:
            return

        current = float(current)
        self.trial.report(current, step=epoch)

        if self.trial.should_prune():
            # extra DB diagnostics for pruned trials
            try:
                self.trial.set_user_attr("pruned_epoch", int(epoch))
                self.trial.set_user_attr("last_val_loss", current)
            except Exception:
                pass

            print(f"⛔ Trial {self.trial.number} pruned at epoch {epoch}.")
            raise optuna.TrialPruned()


class EpochLogger(keras.callbacks.Callback):
    """Prints compact progress every `print_every` epochs."""
    def __init__(self, trial_number: int, print_every: int = 2):
        super().__init__()
        self.trial_number = trial_number
        self.print_every = print_every

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if (epoch + 1) % self.print_every != 0:
            return
        tl   = logs.get("loss", float("nan"))
        vl   = logs.get("val_loss", float("nan"))
        mae  = logs.get("mae", float("nan"))
        vmae = logs.get("val_mae", float("nan"))
        print(
            f"[trial {self.trial_number:03d}] epoch {epoch+1:02d} "
            f"loss={tl:.4f} val_loss={vl:.4f} mae={mae:.4f} val_mae={vmae:.4f}",
            flush=True
        )


def on_trial_end(study: optuna.Study, trial: optuna.trial.FrozenTrial):
    """Compact summary whenever a trial finishes (completed or pruned)."""
    state = trial.state.name
    val = trial.value if trial.value is not None else "NA"
    print("\n=== Trial {:03d} summary ===".format(trial.number), flush=True)
    print(f"Status : {state}", flush=True)
    print(f"Value  : {val}", flush=True)
    print(f"Params : {trial.params}", flush=True)

    if study.best_trial is not None and study.best_trial.value is not None:
        bt = study.best_trial
        print("--- Best so far ------------------------------------", flush=True)
        print(f"Best trial #{bt.number:03d} | val_loss={bt.value:.6f}", flush=True)
        print(f"Best params: {bt.params}", flush=True)
        print("----------------------------------------------------\n", flush=True)

### Step 8 — Evaluate Model Performance (MAE, RMSE, MAPE, Directional Accuracy)

This step evaluates the trained LSTM model on the test set using four key metrics:

- **MAE (Mean Absolute Error):** Average absolute deviation in real price units.  
- **RMSE (Root Mean Squared Error):** Penalises large mistakes more strongly.  
- **MAPE (Mean Absolute Percentage Error):** Scale-independent percentage error.  
- **Directional Accuracy:** Percentage of times the model correctly predicts whether the price will move up or down.

Predictions and true values are first **inverse-scaled** back to real natural-gas prices.  
A plot of predicted vs actual prices is also displayed for visual assessment.

The function returns the full metrics dictionary so it can be saved in Step 9.

In [7]:
def _split_info_snapshot(train_df, val_df, test_df):
    total = len(train_df) + len(val_df) + len(test_df)
    return {
        "train": {"n": len(train_df), "start": str(train_df.date.min().date()), "end": str(train_df.date.max().date())},
        "val":   {"n": len(val_df),   "start": str(val_df.date.min().date()),   "end": str(val_df.date.max().date())},
        "test":  {"n": len(test_df),  "start": str(test_df.date.min().date()),  "end": str(test_df.date.max().date())},
        "fractions": {
            "train": round(len(train_df) / total, 4),
            "val":   round(len(val_df)   / total, 4),
            "test":  round(len(test_df)  / total, 4),
        }
    }

SPLIT_INFO = _split_info_snapshot(train_data, val_data, test_data)


def evaluate_lstm_model(model, X_test, y_test, y_scaler, plot=True, return_series=False):
    """
    Evaluates an LSTM price forecasting model on the test set.
    Computes MAE, RMSE, MAPE, and Directional Accuracy (DA).
    Automatically inverse-scales predictions and returns metrics.
    If return_series=True, also returns (y_true, y_pred) for saving plots.
    """
    y_pred_scaled = model.predict(X_test, verbose=0)
    y_true_scaled = y_test.reshape(-1, 1)

    y_pred = y_scaler.inverse_transform(y_pred_scaled).ravel()
    y_true = y_scaler.inverse_transform(y_true_scaled).ravel()

    mae  = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mape = float(np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-6, None))) * 100)

    # Directional accuracy vs previous TRUE price
    if len(y_true) >= 2:
        prev_true = y_true[:-1]
        true_dir  = np.sign(y_true[1:] - prev_true)
        pred_dir  = np.sign(y_pred[1:] - prev_true)
        directional_accuracy = float((true_dir == pred_dir).mean() * 100)
    else:
        directional_accuracy = float("nan")

    if plot:
        plt.figure(figsize=(10, 4))
        plt.plot(y_true, label="Actual")
        plt.plot(y_pred, label="Predicted")
        plt.title("Next-Day Price Prediction — Test Set")
        plt.legend()
        plt.tight_layout()
        plt.show()

    metrics = {
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape,
        "Directional_Accuracy": directional_accuracy
    }

    if return_series:
        return metrics, y_true, y_pred
    return metrics

### Step 9 — Save the Full Experiment (Model, Scalers, Hyperparameters, Metrics)

This step saves everything needed to fully reproduce the trained model, without storing any raw data.  
It outputs a timestamped folder containing:

- **Trained model** (architecture + weights)  
- **Scalers** for inputs and targets  
- **Hyperparameters** (lookback, units, dropout, learning rate, batch size, etc.)  
- **Training history** (loss/val-loss curves)  
- **Evaluation metrics** (MAE, RMSE, MAPE, etc.)  
- **Data recipe** (features, target, split sizes, date ranges)  
- **Environment info** (Python/TensorFlow versions)  

This ensures the experiment is fully reproducible and the model can be reloaded or deployed later without retraining.

In [8]:
def _to_json_safe(obj):
    """Convert numpy objects to JSON-safe Python types."""
    if isinstance(obj, (np.floating, np.integer)):
        return obj.item()
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj


def export_trial_summary(study: optuna.Study, out_csv: str):
    """
    Export a flat CSV with key metrics as columns (RMSE/MAE/MAPE/DA etc),
    plus params/user_attrs blobs for debugging.
    """
    wanted = [
        # runtime / training
        "duration_sec", "epochs_ran",
        "val_loss_best", "val_mae_best", "train_loss_best",
        # test metrics (real units)
        "test_mae", "test_rmse", "test_mape", "test_directional_accuracy",
        # prune diagnostics (if available)
        "pruned_epoch", "last_val_loss",
        # optional export info (if you set it later)
        "exported_rank", "export_dir",
    ]

    rows = []
    for t in study.trials:
        ua = t.user_attrs or {}

        row = {
            "trial_number": t.number,
            "state": t.state.name,
            "value": t.value,
            "datetime_start": str(t.datetime_start) if t.datetime_start else None,
            "datetime_complete": str(t.datetime_complete) if t.datetime_complete else None,
            "params": json.dumps(t.params, default=str),
        }

        # Flatten selected user attrs into dedicated columns
        for k in wanted:
            row[k] = ua.get(k, None)

        # Keep the full blob too
        row["user_attrs"] = json.dumps(ua, default=str)

        rows.append(row)

    pd.DataFrame(rows).to_csv(out_csv, index=False)
    print(f"Wrote trial summary: {out_csv}")


def save_experiment(
    run_name: str,
    model: keras.Model,
    X_scaler,
    y_scaler,
    features: list,
    target: str,
    lookback: int,
    horizon: int,
    metrics: dict,             # {"test": {...}, "val": {...}}
    hyperparams: dict,         # units, layers, dropout, lr, batch size, etc.
    split_info: dict,          # {"train": {...}, "val": {...}, "test": {...}}
    callbacks_used: list = None,
    notes: str = "",
    save_dir: str = "saved_models",
    out_dir: str | None = None,      # <-- NEW: if provided, save directly here (no timestamp nesting)
    y_true: np.ndarray | None = None,
    y_pred: np.ndarray | None = None,
):
    """
    Saves model + scalers + metadata + pred_vs_actual plot (NO training curves).

    If out_dir is provided, artifacts are written directly into that folder.
    Otherwise creates: <save_dir>/<timestamp>_<run_name>/
    """
    ts = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")

    if out_dir is None:
        run_id = f"{ts}_{run_name}".replace(" ", "_")
        out_dir = os.path.join(save_dir, run_id)
    else:
        run_id = os.path.basename(out_dir.rstrip("/"))

    os.makedirs(out_dir, exist_ok=True)

    # 1) Save model
    model.save(os.path.join(out_dir, "model.keras"))

    # 2) Save scalers
    with open(os.path.join(out_dir, "X_scaler.pkl"), "wb") as f:
        pickle.dump(X_scaler, f)
    with open(os.path.join(out_dir, "y_scaler.pkl"), "wb") as f:
        pickle.dump(y_scaler, f)

    # 3) Environment info
    env = {
        "python": platform.python_version(),
        "tensorflow": tf.__version__,
        "keras": getattr(tf.keras, "__version__", "tf.keras"),
        "numpy": np.__version__,
        "device": tf.test.gpu_device_name() or "CPU/Metal",
        "timestamp_utc": ts,
    }

    # 4) Training configuration
    train_cfg = {
        "optimizer": type(model.optimizer).__name__ if hasattr(model, "optimizer") else None,
        "loss": str(model.loss),
        "callbacks": callbacks_used or [],
    }

    # 5) Data recipe (no raw data)
    data_recipe = {
        "features": features,
        "target": target,
        "lookback": lookback,
        "horizon": horizon,
        "splits": split_info,
    }

    # 6) Metadata JSON (no training history)
    metadata = {
        "run_name": run_name,
        "run_id": run_id,
        "out_dir": out_dir,
        "model_path": "model.keras",
        "artifacts": {
            "X_scaler": "X_scaler.pkl",
            "y_scaler": "y_scaler.pkl",
            "pred_vs_actual": "pred_vs_actual.png" if (y_true is not None and y_pred is not None) else None,
        },
        "hyperparameters": hyperparams,
        "training_config": train_cfg,
        "metrics": {k: {m: _to_json_safe(v) for m, v in d.items()} for k, d in metrics.items()},
        "data_recipe": data_recipe,
        "environment": env,
        "notes": notes,
    }

    with open(os.path.join(out_dir, "metadata.json"), "w") as f:
        json.dump(metadata, f, indent=2)

    # 7) Pred vs Actual plot
    if y_true is not None and y_pred is not None:
        plt.figure(figsize=(10, 4))
        plt.plot(y_true, label="Actual")
        plt.plot(y_pred, label="Predicted")
        plt.title(f"{run_name} — Predicted vs Actual (Test Set)")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, "pred_vs_actual.png"), dpi=150)
        plt.close()

    print(f"Saved experiment to: {out_dir}")
    return out_dir


def _print_trial_header(trial):
    print("\n" + "=" * 72, flush=True)
    print(f"🔎 Starting trial {trial.number}", flush=True)
    print("=" * 72, flush=True)


def objective(trial):
    start_time = time.time()
    _print_trial_header(trial)

    try:
        # ---- sample sequence windowing ----
        lookback   = trial.suggest_categorical("lookback", [45, 60, 75])
        batch_size = trial.suggest_categorical("batch_size", [32, 64])
        print(f"[trial {trial.number:03d}] lookback={lookback}, batch_size={batch_size}")

        # ---- re-window (scaled arrays) ----
        Xtr, ytr = make_windows(X_train_raw, y_train_raw, lookback=lookback, horizon=HORIZON)
        Xva, yva = make_windows(X_val_raw,   y_val_raw,   lookback=lookback, horizon=HORIZON)
        Xte, yte = make_windows(X_test_raw,  y_test_raw,  lookback=lookback, horizon=HORIZON)
        steps, feats = Xtr.shape[1], Xtr.shape[2]

        # ---- build model ----
        model, lstm_units, lr, dropout, rdrop, dense_units = build_lstm_model_from_trial(trial, steps, feats)

        # ---- callbacks ----
        callbacks = [
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_loss"),
            keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, monitor="val_loss"),
            ManualPruningCallback(trial),
            EpochLogger(trial.number, print_every=2),
        ]

        # ---- train ----
        history = model.fit(
            Xtr, ytr,
            validation_data=(Xva, yva),
            epochs=50,
            batch_size=batch_size,
            verbose=0,
            callbacks=callbacks
        )

        # ---- best-in-run stats ----
        val_loss_best    = float(np.min(history.history["val_loss"]))
        val_mae_best     = float(np.min(history.history.get("val_mae", [np.nan])))
        train_loss_best  = float(np.min(history.history.get("loss", [np.nan])))
        epochs_ran       = int(len(history.history["val_loss"]))
        duration_sec     = float(time.time() - start_time)

        # ---- evaluate on test (single predict) ----
        test_metrics, _, _ = evaluate_lstm_model(model, Xte, yte, y_scaler, plot=False, return_series=True)

        # ---- store key metrics in Optuna DB (user_attrs) ----
        trial.set_user_attr("duration_sec", duration_sec)
        trial.set_user_attr("epochs_ran", epochs_ran)
        trial.set_user_attr("val_loss_best", val_loss_best)
        trial.set_user_attr("val_mae_best", val_mae_best)
        trial.set_user_attr("train_loss_best", train_loss_best)

        # IMPORTANT: map test metrics to stable snake_case keys for CSV
        trial.set_user_attr("test_mae", float(test_metrics["MAE"]))
        trial.set_user_attr("test_rmse", float(test_metrics["RMSE"]))
        trial.set_user_attr("test_mape", float(test_metrics["MAPE"]))
        trial.set_user_attr("test_directional_accuracy", float(test_metrics["Directional_Accuracy"]))

        # also store a few config things
        trial.set_user_attr("lstm_units", lstm_units)
        trial.set_user_attr("dense_units", int(dense_units))
        trial.set_user_attr("dropout", float(dropout))
        trial.set_user_attr("recurrent_dropout", float(rdrop))
        trial.set_user_attr("lr", float(lr))
        trial.set_user_attr("lookback", int(lookback))
        trial.set_user_attr("batch_size", int(batch_size))

        print(f"[trial {trial.number:03d}] ✅ COMPLETED in {duration_sec:.1f}s | best val_loss={val_loss_best:.4f}")

        # objective value for Optuna
        return val_loss_best

    except optuna.TrialPruned:
        print(f"[trial {trial.number:03d}] ⛔ PRUNED")
        raise

    finally:
        K.clear_session()


# ---- Optuna study (SQLite-backed) ----
storage = f"sqlite:///{DB_PATH}"

study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=storage,
    load_if_exists=True,
    direction="minimize",
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_warmup_steps=5),
)

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
optuna.logging.set_verbosity(optuna.logging.INFO)

study.optimize(objective, n_trials=300, gc_after_trial=True, callbacks=[on_trial_end])

# CSV now includes flattened metrics columns (rmse/mae/mape/da, etc.)
export_trial_summary(study, TRIAL_SUMMARY_CSV)

# ---- Save top-K only ----
TOP_K = 10
completed = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE and t.value is not None]
top_trials = sorted(completed, key=lambda t: t.value)[:TOP_K]

print(f"Saving top {len(top_trials)} models to: {TOPMODELS_DIR}")

for rank, t in enumerate(top_trials, start=1):
    lookback = int(t.params.get("lookback", 60))
    batch_size = int(t.params.get("batch_size", 64))

    Xtr, ytr = make_windows(X_train_raw, y_train_raw, lookback=lookback, horizon=HORIZON)
    Xva, yva = make_windows(X_val_raw,   y_val_raw,   lookback=lookback, horizon=HORIZON)
    Xte, yte = make_windows(X_test_raw,  y_test_raw,  lookback=lookback, horizon=HORIZON)
    steps, feats = Xtr.shape[1], Xtr.shape[2]

    fixed = optuna.trial.FixedTrial(t.params)
    model, lstm_units, lr, dropout, rdrop, dense_units = build_lstm_model_from_trial(fixed, steps, feats)

    callbacks = [
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_loss"),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, monitor="val_loss"),
    ]

    model.fit(
        Xtr, ytr,
        validation_data=(Xva, yva),
        epochs=50,
        batch_size=batch_size,
        verbose=0,
        callbacks=callbacks
    )

    test_metrics, y_true, y_pred = evaluate_lstm_model(model, Xte, yte, y_scaler, plot=False, return_series=True)

    out_dir = os.path.join(TOPMODELS_DIR, f"rank_{rank:02d}_trial_{t.number:04d}")
    run_title = f"{EXPERIMENT_NAME} | trial {t.number:04d} | rank {rank:02d}"
    os.makedirs(out_dir, exist_ok=True)

    metrics_to_save = {"val": {"loss": float(t.value)}, "test": test_metrics}

    hparams = dict(t.params)
    hparams.update({
        "lstm_units": lstm_units,
        "dense_units": dense_units,
        "dropout": dropout,
        "recurrent_dropout": rdrop,
        "lr": lr,
        "batch_size": batch_size,
        "lookback": lookback,
        "horizon": HORIZON,
    })

    save_experiment(
        out_dir=out_dir,            # <-- explicit directory (no extra timestamp nesting)
        run_name=run_title,
        model=model,
        X_scaler=X_scaler,
        y_scaler=y_scaler,
        features=FEATURES,
        target=TARGET,
        lookback=lookback,
        horizon=HORIZON,
        metrics=metrics_to_save,
        hyperparams=hparams,
        split_info=SPLIT_INFO,
        callbacks_used=["EarlyStopping", "ReduceLROnPlateau"],
        notes=f"Top-{TOP_K} export from study {STUDY_NAME}.",
        y_true=y_true,
        y_pred=y_pred,
    )

    K.clear_session()

print("Best value (val_loss):", study.best_value)
print("Best params:", study.best_params)

[I 2026-01-14 03:52:27,910] A new study created in RDB with name: 20260114-035227_PriceOnly_lstm



🔎 Starting trial 0
[trial 000] lookback=60, batch_size=32
[trial 000] epoch 02 loss=0.2164 val_loss=2.2765 mae=0.2099 val_mae=1.0660
[trial 000] epoch 04 loss=0.1925 val_loss=1.6769 mae=0.1825 val_mae=0.8940
[trial 000] epoch 06 loss=0.1782 val_loss=1.3481 mae=0.1678 val_mae=0.7908
[trial 000] epoch 08 loss=0.1678 val_loss=1.1766 mae=0.1577 val_mae=0.7323
[trial 000] epoch 10 loss=0.1589 val_loss=1.0613 mae=0.1488 val_mae=0.6926
[trial 000] epoch 12 loss=0.1512 val_loss=0.9697 mae=0.1412 val_mae=0.6600
[trial 000] epoch 14 loss=0.1449 val_loss=0.8962 mae=0.1351 val_mae=0.6328
[trial 000] epoch 16 loss=0.1402 val_loss=0.8385 mae=0.1308 val_mae=0.6099
[trial 000] epoch 18 loss=0.1361 val_loss=0.7910 mae=0.1274 val_mae=0.5895
[trial 000] epoch 20 loss=0.1323 val_loss=0.7483 mae=0.1243 val_mae=0.5708
[trial 000] epoch 22 loss=0.1285 val_loss=0.7077 mae=0.1206 val_mae=0.5521
[trial 000] epoch 24 loss=0.1243 val_loss=0.6669 mae=0.1160 val_mae=0.5330
[trial 000] epoch 26 loss=0.1203 val_loss

[I 2026-01-14 03:53:20,073] Trial 0 finished with value: 0.31191906332969666 and parameters: {'lookback': 60, 'batch_size': 32, 'layers': 3, 'units3_idx': 1, 'dropout': 0.0, 'lr': 0.00020589728197687926}. Best is trial 0 with value: 0.31191906332969666.


[trial 000] ✅ COMPLETED in 51.7s | best val_loss=0.3119

=== Trial 000 summary ===
Status : COMPLETE
Value  : 0.31191906332969666
Params : {'lookback': 60, 'batch_size': 32, 'layers': 3, 'units3_idx': 1, 'dropout': 0.0, 'lr': 0.00020589728197687926}
--- Best so far ------------------------------------
Best trial #000 | val_loss=0.311919
Best params: {'lookback': 60, 'batch_size': 32, 'layers': 3, 'units3_idx': 1, 'dropout': 0.0, 'lr': 0.00020589728197687926}
----------------------------------------------------


🔎 Starting trial 1
[trial 001] lookback=75, batch_size=32
[trial 001] epoch 02 loss=0.1645 val_loss=0.8842 mae=0.1498 val_mae=0.6537
[trial 001] epoch 04 loss=0.1366 val_loss=0.7150 mae=0.1253 val_mae=0.5837
[trial 001] epoch 06 loss=0.1186 val_loss=0.6194 mae=0.1131 val_mae=0.5349
[trial 001] epoch 08 loss=0.1028 val_loss=0.4599 mae=0.1071 val_mae=0.4545
[trial 001] epoch 10 loss=0.0915 val_loss=0.3019 mae=0.1008 val_mae=0.3689
[trial 001] epoch 12 loss=0.0844 val_loss=0.2240 

[I 2026-01-14 03:53:42,948] Trial 1 finished with value: 0.20972992479801178 and parameters: {'lookback': 75, 'batch_size': 32, 'layers': 2, 'units2_idx': 2, 'dropout': 0.0, 'lr': 0.0005748924681991981}. Best is trial 1 with value: 0.20972992479801178.


...


=== Trial 297 summary ===
Status : COMPLETE
Value  : 0.09146571159362793
Params : {'lookback': 45, 'batch_size': 64, 'layers': 1, 'units1': 64, 'dropout': 0.0, 'lr': 0.0028097129209396715}
--- Best so far ------------------------------------
Best trial #225 | val_loss=0.090967
Best params: {'lookback': 45, 'batch_size': 64, 'layers': 1, 'units1': 64, 'dropout': 0.0, 'lr': 0.0023573059965274635}
----------------------------------------------------


🔎 Starting trial 298
[trial 298] lookback=45, batch_size=64
[trial 298] epoch 02 loss=0.1276 val_loss=0.4276 mae=0.1172 val_mae=0.4416
[trial 298] epoch 04 loss=0.1005 val_loss=0.2039 mae=0.1024 val_mae=0.3031


[I 2026-01-14 04:18:20,208] Trial 298 pruned. 


⛔ Trial 298 pruned at epoch 5.
[trial 298] ⛔ PRUNED

=== Trial 298 summary ===
Status : PRUNED
Value  : 0.11340643465518951
Params : {'lookback': 45, 'batch_size': 64, 'layers': 1, 'units1': 64, 'dropout': 0.0, 'lr': 0.002830528569315008}
--- Best so far ------------------------------------
Best trial #225 | val_loss=0.090967
Best params: {'lookback': 45, 'batch_size': 64, 'layers': 1, 'units1': 64, 'dropout': 0.0, 'lr': 0.0023573059965274635}
----------------------------------------------------


🔎 Starting trial 299
[trial 299] lookback=45, batch_size=64
[trial 299] epoch 02 loss=0.1244 val_loss=0.4571 mae=0.1126 val_mae=0.4545
[trial 299] epoch 04 loss=0.1022 val_loss=0.2650 mae=0.0980 val_mae=0.3469


[I 2026-01-14 04:18:24,514] Trial 299 pruned. 


⛔ Trial 299 pruned at epoch 5.
[trial 299] ⛔ PRUNED

=== Trial 299 summary ===
Status : PRUNED
Value  : 0.16052724421024323
Params : {'lookback': 45, 'batch_size': 64, 'layers': 1, 'units1': 64, 'dropout': 0.0, 'lr': 0.0019447363413152069}
--- Best so far ------------------------------------
Best trial #225 | val_loss=0.090967
Best params: {'lookback': 45, 'batch_size': 64, 'layers': 1, 'units1': 64, 'dropout': 0.0, 'lr': 0.0023573059965274635}
----------------------------------------------------

Wrote trial summary: saved_results/20260114-035227_PriceOnly_lstm/trial_summary.csv
Saving top 10 models to: saved_results/20260114-035227_PriceOnly_lstm/top_models
Saved experiment to: saved_results/20260114-035227_PriceOnly_lstm/top_models/rank_01_trial_0225
Saved experiment to: saved_results/20260114-035227_PriceOnly_lstm/top_models/rank_02_trial_0174
Saved experiment to: saved_results/20260114-035227_PriceOnly_lstm/top_models/rank_03_trial_0123
Saved experiment to: saved_results/20260114-